### 165 calculation target loyalty feature
targetとなっているloyaltyを計算している期間に絞ったfeatureを作成する

In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Load

In [28]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = reduce_mem_usage(df_hist)

100%|██████████| 3/3 [00:12<00:00,  4.01s/it]


Mem. usage decreased to 2554.26 Mb (0.0% reduction)


In [29]:
# term
col_month = 'month_lag'
# hist_latest3 = df_hist[df_hist[col_month]>=-2]
df = df_hist[df_hist[col_month]<-2]
# df = df_new

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

auth1_cat1 = auth1[auth1.category_1==1]
auth1_cat0 = auth1[auth1.category_1==0]
auth0_cat1 = auth0[auth0.category_1==1]
auth0_cat0 = auth0[auth0.category_1==0]
del auth1, auth0
gc.collect()

133

### Null Count

In [30]:
last_type = 'lag0_-2'
last_type = 'lag-3_-13'
null_cols = ['category_2', 'installments', 'merchant_id']

df_list = [auth1_cat1, auth1_cat0, auth0_cat1, auth0_cat0]

# category_2
col = 'category_2'
for df in df_list:
    df[col].fillna('NA', inplace=True)
fname = 'auth1_cat0'
auth1_cat0[f'{col}_null_cnt'] = (auth1_cat0[col]=='NA')
fname = 'auth0_cat0'
auth0_cat0[f'{col}_null_cnt'] = (auth0_cat0[col]=='NA')

# installments
col = 'installments'
fname = 'auth1_cat0'
auth1_cat0[f'{col}_isnull'] = (auth1_cat0[col].isnull())
fname = 'auth1_cat1'
auth1_cat1[f'{col}_isnull'] = (auth1_cat1[col].isnull())
fname = 'auth0_cat0'
auth0_cat0[f'{col}_isnull'] = (auth0_cat0[col].isnull())
fname = 'auth0_cat1'
auth0_cat1[f'{col}_isnull'] = (auth0_cat1[col].isnull())

# merchant_id
col = 'merchant_id'
fname = 'auth1_cat0'
auth1_cat0[f'{col}_isnull'] = (auth1_cat0[col].isnull())
fname = 'auth1_cat1'
auth1_cat1[f'{col}_isnull'] = (auth1_cat1[col].isnull())
fname = 'auth0_cat0'
auth0_cat0[f'{col}_isnull'] = (auth0_cat0[col].isnull())
fname = 'auth0_cat1'
auth0_cat1[f'{col}_isnull'] = (auth0_cat1[col].isnull())

In [31]:
last_type = 'lag0_-2'
last_type = 'lag-3_-13'
df_list = [auth1_cat1, auth1_cat0, auth0_cat1, auth0_cat0]
fname_list = [f'{last_type}_auth1_cat1', f'{last_type}_auth1_cat0', f'{last_type}_auth0_cat1', f'{last_type}_auth0_cat0']
train_list = []
test_list = []

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


def feat_agg(df, fname):

    null_cols = [col for col in df.columns if col.count('isnull')]
    
    aggs = {}
    
    # category_1==1のときcategory_2 = Null
    if fname.count('cat0'):
        cat_list = ['category_2']
        df = get_dummies(df=df, cat_list=cat_list)
        category_2_list = [col for col in df.columns if col.count('category_2')]
        for cat in category_2_list:
            aggs[cat] = ['mean', 'sum']
            
    # isnull feature
    for col in null_cols:
        aggs[col] = ['mean', 'sum']
    
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    df['unix_date'] =  df['purchase_date'].map(lambda x: int(x.timestamp()) )
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['purchase_amount_over'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum','max', 'min','mean', 'std']
    aggs['unix_date'] = ['max','min']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    aggs['card_id'] = ['size']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
    
    df_agg[f'{fname}_term'] = (df_agg[f'{fname}_unix_date_max'] - df_agg[f'{fname}_unix_date_min']).astype('int64')
    df_agg[f'{fname}_freq_per_90days'] = df_agg[f'{fname}_card_id_size'] / 90
    df_agg[f'{fname}_amount_per_90days'] = df_agg[f'{fname}_purchase_amount_sum'] / 90
    df_agg[f'{fname}_installments_per_90days'] = df_agg[f'{fname}_installments_sum'] / 90
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / df_agg[f'{fname}_installments_sum']
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / df_agg[f'{fname}_installments_mean']
    
    feat_train = df_train.merge(df_agg, on=key, how='left')
    feat_test = df_test.merge(df_agg, on=key, how='left')
    del df_agg
    gc.collect()
    
    print(fname)
    display(feat_train.head())
    
    train_list.append(feat_train)
    test_list.append(feat_test)
    
for df, fname in zip(df_list, fname_list):
    feat_agg(df, fname)

lag-3_-13_auth1_cat1


,card_id,first_active_month,feature_1,feature_2,feature_3,target,lag-3_-13_auth1_cat1_installments_isnull_mean,lag-3_-13_auth1_cat1_installments_isnull_sum,lag-3_-13_auth1_cat1_merchant_id_isnull_mean,lag-3_-13_auth1_cat1_merchant_id_isnull_sum,lag-3_-13_auth1_cat1_month_lag_mean,lag-3_-13_auth1_cat1_month_lag_std,lag-3_-13_auth1_cat1_yyyy_week_nunique,lag-3_-13_auth1_cat1_purchase_amount_sum,lag-3_-13_auth1_cat1_purchase_amount_max,lag-3_-13_auth1_cat1_purchase_amount_min,lag-3_-13_auth1_cat1_purchase_amount_mean,lag-3_-13_auth1_cat1_purchase_amount_over_sum,lag-3_-13_auth1_cat1_purchase_amount_over_max,lag-3_-13_auth1_cat1_purchase_amount_over_min,lag-3_-13_auth1_cat1_purchase_amount_over_mean,lag-3_-13_auth1_cat1_installments_sum,lag-3_-13_auth1_cat1_installments_max,lag-3_-13_auth1_cat1_installments_min,lag-3_-13_auth1_cat1_installments_mean,lag-3_-13_auth1_cat1_installments_std,lag-3_-13_auth1_cat1_unix_date_max,lag-3_-13_auth1_cat1_unix_date_min,lag-3_-13_auth1_cat1_purchase_date_max,lag-3_-13_auth1_cat1_purchase_date_min,lag-3_-13_auth1_cat1_month_diff_mean,lag-3_-13_auth1_cat1_month_diff_std,lag-3_-13_auth1_cat1_card_id_size,lag-3_-13_auth1_cat1_merchant_id_nunique,lag-3_-13_auth1_cat1_merchant_category_id_nunique,lag-3_-13_auth1_cat1_term,lag-3_-13_auth1_cat1_freq_per_90days,lag-3_-13_auth1_cat1_amount_per_90days,lag-3_-13_auth1_cat1_installments_per_90days,lag-3_-13_auth1_cat1_amount_per_installments_sum,lag-3_-13_auth1_cat1_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,False,False,0.0,0.0,-9.875,3.054754,12.0,5.894531,2.0,0.006020,0.245605,5.378906,5.378906,0.0,0.224121,49.0,10.0,1.0,2.041016,2.009766,1.508668e+09,1.484054e+09,2017-10-22 10:27:43,2017-01-10 13:17:29,400.625,93.708063,24.0,12.0,12.0,24613814.0,0.266667,0.065491,0.544434,0.120300,0.120361
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,False,False,0.0,0.0,-3.400,0.547723,4.0,3.222656,2.0,0.176270,0.644531,2.193359,2.193359,0.0,0.438721,12.0,3.0,1.0,2.400391,0.894531,1.511523e+09,1.507657e+09,2017-11-24 11:32:02,2017-10-10 17:33:48,175.200,22.015903,5.0,5.0,5.0,3866294.0,0.055556,0.035797,0.133301,0.268555,0.268555
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,False,False,0.0,0.0,-3.000,0.000000,1.0,2.892578,2.0,0.892578,1.446289,2.933594,2.933594,0.0,1.466797,6.0,3.0,3.0,3.000000,0.000000,1.511610e+09,1.511534e+09,2017-11-25 11:41:03,2017-11-24 14:39:12,156.500,0.707107,2.0,2.0,2.0,75711.0,0.022222,0.032135,0.066650,0.482178,0.482178


lag-3_-13_auth1_cat0


,card_id,first_active_month,feature_1,feature_2,feature_3,target,lag-3_-13_auth1_cat0_category_2_null_cnt_mean,lag-3_-13_auth1_cat0_category_2_null_cnt_sum,lag-3_-13_auth1_cat0_dummie_category_2_1.0_mean,lag-3_-13_auth1_cat0_dummie_category_2_1.0_sum,lag-3_-13_auth1_cat0_dummie_category_2_2.0_mean,lag-3_-13_auth1_cat0_dummie_category_2_2.0_sum,lag-3_-13_auth1_cat0_dummie_category_2_3.0_mean,lag-3_-13_auth1_cat0_dummie_category_2_3.0_sum,lag-3_-13_auth1_cat0_dummie_category_2_4.0_mean,lag-3_-13_auth1_cat0_dummie_category_2_4.0_sum,lag-3_-13_auth1_cat0_dummie_category_2_5.0_mean,lag-3_-13_auth1_cat0_dummie_category_2_5.0_sum,lag-3_-13_auth1_cat0_dummie_category_2_NA_mean,lag-3_-13_auth1_cat0_dummie_category_2_NA_sum,lag-3_-13_auth1_cat0_installments_isnull_mean,lag-3_-13_auth1_cat0_installments_isnull_sum,lag-3_-13_auth1_cat0_merchant_id_isnull_mean,lag-3_-13_auth1_cat0_merchant_id_isnull_sum,lag-3_-13_auth1_cat0_month_lag_mean,lag-3_-13_auth1_cat0_month_lag_std,lag-3_-13_auth1_cat0_yyyy_week_nunique,lag-3_-13_auth1_cat0_purchase_amount_sum,lag-3_-13_auth1_cat0_purchase_amount_max,lag-3_-13_auth1_cat0_purchase_amount_min,lag-3_-13_auth1_cat0_purchase_amount_mean,lag-3_-13_auth1_cat0_purchase_amount_over_sum,lag-3_-13_auth1_cat0_purchase_amount_over_max,lag-3_-13_auth1_cat0_purchase_amount_over_min,lag-3_-13_auth1_cat0_purchase_amount_over_mean,lag-3_-13_auth1_cat0_installments_sum,lag-3_-13_auth1_cat0_installments_max,lag-3_-13_auth1_cat0_installments_min,lag-3_-13_auth1_cat0_installments_mean,lag-3_-13_auth1_cat0_installments_std,lag-3_-13_auth1_cat0_unix_date_max,lag-3_-13_auth1_cat0_unix_date_min,lag-3_-13_auth1_cat0_purchase_date_max,lag-3_-13_auth1_cat0_purchase_date_min,lag-3_-13_auth1_cat0_month_diff_mean,lag-3_-13_auth1_cat0_month_diff_std,lag-3_-13_auth1_cat0_card_id_size,lag-3_-13_auth1_cat0_merchant_id_nunique,lag-3_-13_auth1_cat0_merchant_category_id_nunique,lag-3_-13_auth1_cat0_term,lag-3_-13_auth1_cat0_freq_per_90days,lag-3_-13_auth1_cat0_amount_per_90days,lag-3_-13_auth1_cat0_installments_per_90days,lag-3_-13_auth1_cat0_amount_per_installments_sum,lag-3_-13_auth1_cat0_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,0.0,0.0,1.000000,148.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,False,False,0.0,0.0,-5.594595,1.432556,23.0,11.078125,0.439697,0.007523,0.074890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.512000e+09,1.498573e+09,2017-11-30 00:00:00,2017-06-27 14:18:08,245.283784,45.645639,148.0,59.0,26.0,13426912.0,1.644444,0.123108,0.000000,inf,inf
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,0.0,0.0,1.000000,203.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,False,False,0.0,0.0,-6.886700,2.566342,41.0,27.296875,1.620117,0.006172,0.134399,0.0,0.0,0.0,0.0,319.0,10.0,1.0,1.571289,1.495117,1.509217e+09,1.483720e+09,2017-10-28 18:50:13,2017-01-06 16:29:42,314.891626,78.702083,203.0,101.0,41.0,25496431.0,2.255556,0.303223,3.544922,0.085571,0.085510
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,0.0,0.0,0.108108,4.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.891892,33.0,0.0,0.0,False,False,0.0,0.0,-9.297297,3.143466,21.0,2.009766,0.114136,0.016785,0.054321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.509299e+09,1.484123e+09,2017-10-29 17:43:31,2017-01-11 08:21:22,354.243243,94.879574,37.0,11.0,7.0,25176129.0,0.411111,0.022324,0.000000,inf,inf
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,0.0,0.0,0.227273,10.0,0.0,0.0,0.0,0.0,0.772727,34.0,0.000000,0.0,0.0,0.0,False,False,0.0,0.0,-4.159091,0.568277,9.0,2.185547,0.282227,0.007507,0.049652,0.0,0.0,0.0,0.0,44.0,2.0,0.0,1.000000,0.215698,1.511557e+09,1.506443e+09,2017-11-24 20:59:18,2017-09-26 16:22:21,198.318182,15.100859,44.0,29.0,17.0,5114217.0,0.488889,0.024277,0.488770,0.049683,0.049652
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.947368,18.0,0.052632,1.0,0.0,0.0,False,False,0.0,0.0,-3.000000,0.000000,4.0,1.984375,0.320312,0.038300,0.104431,0.0,0.0,0.0,0.0,19.0,1.0,1.0,1.000000,0.000000,1.511914e+09,1.510445e+09,2017-11-2

lag-3_-13_auth0_cat1


,card_id,first_active_month,feature_1,feature_2,feature_3,target,lag-3_-13_auth0_cat1_installments_isnull_mean,lag-3_-13_auth0_cat1_installments_isnull_sum,lag-3_-13_auth0_cat1_merchant_id_isnull_mean,lag-3_-13_auth0_cat1_merchant_id_isnull_sum,lag-3_-13_auth0_cat1_month_lag_mean,lag-3_-13_auth0_cat1_month_lag_std,lag-3_-13_auth0_cat1_yyyy_week_nunique,lag-3_-13_auth0_cat1_purchase_amount_sum,lag-3_-13_auth0_cat1_purchase_amount_max,lag-3_-13_auth0_cat1_purchase_amount_min,lag-3_-13_auth0_cat1_purchase_amount_mean,lag-3_-13_auth0_cat1_purchase_amount_over_sum,lag-3_-13_auth0_cat1_purchase_amount_over_max,lag-3_-13_auth0_cat1_purchase_amount_over_min,lag-3_-13_auth0_cat1_purchase_amount_over_mean,lag-3_-13_auth0_cat1_installments_sum,lag-3_-13_auth0_cat1_installments_max,lag-3_-13_auth0_cat1_installments_min,lag-3_-13_auth0_cat1_installments_mean,lag-3_-13_auth0_cat1_installments_std,lag-3_-13_auth0_cat1_unix_date_max,lag-3_-13_auth0_cat1_unix_date_min,lag-3_-13_auth0_cat1_purchase_date_max,lag-3_-13_auth0_cat1_purchase_date_min,lag-3_-13_auth0_cat1_month_diff_mean,lag-3_-13_auth0_cat1_month_diff_std,lag-3_-13_auth0_cat1_card_id_size,lag-3_-13_auth0_cat1_merchant_id_nunique,lag-3_-13_auth0_cat1_merchant_category_id_nunique,lag-3_-13_auth0_cat1_term,lag-3_-13_auth0_cat1_freq_per_90days,lag-3_-13_auth0_cat1_amount_per_90days,lag-3_-13_auth0_cat1_installments_per_90days,lag-3_-13_auth0_cat1_amount_per_installments_sum,lag-3_-13_auth0_cat1_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,False,False,0.0,0.0,-6.0,2.828427,2.0,1.015625,1.009766,0.00602,0.507812,0.0,0.0,0.0,0.0,11.0,10.0,1.0,5.5,6.363281,1.505683e+09,1.494244e+09,2017-09-17 21:22:50,2017-05-08 11:43:11,291.0,93.338095,2.0,2.0,2.0,11439579.0,0.022222,0.011284,0.122192,0.092346,0.092346
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


lag-3_-13_auth0_cat0


,card_id,first_active_month,feature_1,feature_2,feature_3,target,lag-3_-13_auth0_cat0_category_2_null_cnt_mean,lag-3_-13_auth0_cat0_category_2_null_cnt_sum,lag-3_-13_auth0_cat0_dummie_category_2_1.0_mean,lag-3_-13_auth0_cat0_dummie_category_2_1.0_sum,lag-3_-13_auth0_cat0_dummie_category_2_2.0_mean,lag-3_-13_auth0_cat0_dummie_category_2_2.0_sum,lag-3_-13_auth0_cat0_dummie_category_2_3.0_mean,lag-3_-13_auth0_cat0_dummie_category_2_3.0_sum,lag-3_-13_auth0_cat0_dummie_category_2_4.0_mean,lag-3_-13_auth0_cat0_dummie_category_2_4.0_sum,lag-3_-13_auth0_cat0_dummie_category_2_5.0_mean,lag-3_-13_auth0_cat0_dummie_category_2_5.0_sum,lag-3_-13_auth0_cat0_dummie_category_2_NA_mean,lag-3_-13_auth0_cat0_dummie_category_2_NA_sum,lag-3_-13_auth0_cat0_installments_isnull_mean,lag-3_-13_auth0_cat0_installments_isnull_sum,lag-3_-13_auth0_cat0_merchant_id_isnull_mean,lag-3_-13_auth0_cat0_merchant_id_isnull_sum,lag-3_-13_auth0_cat0_month_lag_mean,lag-3_-13_auth0_cat0_month_lag_std,lag-3_-13_auth0_cat0_yyyy_week_nunique,lag-3_-13_auth0_cat0_purchase_amount_sum,lag-3_-13_auth0_cat0_purchase_amount_max,lag-3_-13_auth0_cat0_purchase_amount_min,lag-3_-13_auth0_cat0_purchase_amount_mean,lag-3_-13_auth0_cat0_purchase_amount_over_sum,lag-3_-13_auth0_cat0_purchase_amount_over_max,lag-3_-13_auth0_cat0_purchase_amount_over_min,lag-3_-13_auth0_cat0_purchase_amount_over_mean,lag-3_-13_auth0_cat0_installments_sum,lag-3_-13_auth0_cat0_installments_max,lag-3_-13_auth0_cat0_installments_min,lag-3_-13_auth0_cat0_installments_mean,lag-3_-13_auth0_cat0_installments_std,lag-3_-13_auth0_cat0_unix_date_max,lag-3_-13_auth0_cat0_unix_date_min,lag-3_-13_auth0_cat0_purchase_date_max,lag-3_-13_auth0_cat0_purchase_date_min,lag-3_-13_auth0_cat0_month_diff_mean,lag-3_-13_auth0_cat0_month_diff_std,lag-3_-13_auth0_cat0_card_id_size,lag-3_-13_auth0_cat0_merchant_id_nunique,lag-3_-13_auth0_cat0_merchant_category_id_nunique,lag-3_-13_auth0_cat0_term,lag-3_-13_auth0_cat0_freq_per_90days,lag-3_-13_auth0_cat0_amount_per_90days,lag-3_-13_auth0_cat0_installments_per_90days,lag-3_-13_auth0_cat0_amount_per_installments_sum,lag-3_-13_auth0_cat0_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.090909,1.0,-4.909091,1.375103,7.0,1.088867,0.314941,0.009026,0.098938,0.000000,0.000000,0.0,0.000000,4.0,1.0,0.0,0.363525,0.504395,1.509840e+09,1.500131e+09,2017-11-05 00:00:00,2017-07-15 14:58:10,228.090909,39.300012,11.0,10.0,8.0,9709310.0,0.122222,0.012100,0.044434,0.272217,0.272217
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,0.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000000,0.0,-5.000000,2.380476,6.0,5.363281,2.000000,0.225464,0.766113,2.689453,2.689453,0.0,0.384277,29.0,10.0,1.0,4.144531,3.132812,1.507248e+09,1.488576e+09,2017-10-06 00:07:48,2017-03-03 21:27:17,257.000000,78.260250,7.0,6.0,6.0,18672031.0,0.077778,0.059601,0.322266,0.184937,0.184814
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,False,False,0.000000,0.0,-11.000000,1.414214,2.0,0.154785,0.109375,0.045471,0.077393,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.492801e+09,1.487878e+09,2017-04-21 19:00:14,2017-02-23 19:32:09,402.500000,40.305087,2.0,1.0,1.0,4922885.0,0.022222,0.001719,0.000000,inf,inf
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# impute

def impute_agg(df):
    for col in df.columns:
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df
        
for train, test in zip(train_list, test_list):
    try:
        train.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
        test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month'], axis=1, inplace=True)
    except KeyError:
        pass
    
    try:
        train.set_index(key, inplace=True)
        test.set_index(key, inplace=True)
    except KeyError:
        pass
    
    df_train = df_train.join(train)
    df_test = df_test.join(test)

df_train = impute_agg(df_train)
df_test = impute_agg(df_test)

In [33]:
last_type = 'lag0_-2'
last_type = 'lag-3_-13'

df_train['first_active_month'] =  pd.to_datetime(df_train['first_active_month'])
df_test['first_active_month'] =  pd.to_datetime(df_test['first_active_month'])

# 最終のcat0, auth1までの期間
df_train[f'{last_type}_auth1_cat0_elapsed_time_from_first'] = (df_train[f'{last_type}_auth1_cat0_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_auth1_cat0_elapsed_time_from_first']  = (df_test[f'{last_type}_auth1_cat0_purchase_date_max'] - df_test[f'first_active_month']).dt.days

# 最終のcat1, auth1までの期間
df_train[f'{last_type}_cat0_elapsed_time_from_first'] = (df_train[f'{last_type}_auth1_cat0_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat0_elapsed_time_from_first']  = (df_test[f'{last_type}_auth1_cat0_purchase_date_max'] - df_test[f'first_active_month']).dt.days
df_train[f'{last_type}_cat1_elapsed_time_from_first'] = (df_train[f'{last_type}_auth1_cat1_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat1_elapsed_time_from_first']  = (df_test[f'{last_type}_auth1_cat1_purchase_date_max'] - df_test[f'first_active_month']).dt.days

# {last_type}の初回購買までの期間
df_train[f'{last_type}_cat0_first_buy'] = (df_train[f'{last_type}_auth1_cat0_purchase_date_min'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat0_first_buy']  = (df_test[f'{last_type}_auth1_cat0_purchase_date_min'] - df_test[f'first_active_month']).dt.days
df_train[f'{last_type}_cat1_first_buy'] = (df_train[f'{last_type}_auth1_cat1_purchase_date_min'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat1_first_buy']  = (df_test[f'{last_type}_auth1_cat1_purchase_date_min'] - df_test[f'first_active_month']).dt.days

# 20180501までの期間を各データセットパターンで
df_train[f'{last_type}_auth1_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth1_cat0_purchase_date_max']).dt.days
df_train[f'{last_type}_auth1_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth1_cat1_purchase_date_max']).dt.days
df_train[f'{last_type}_auth0_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth0_cat0_purchase_date_max']).dt.days
df_train[f'{last_type}_auth0_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth0_cat1_purchase_date_max']).dt.days
df_test[f'{last_type}_auth1_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth1_cat0_purchase_date_max']).dt.days
df_test[f'{last_type}_auth1_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth1_cat1_purchase_date_max']).dt.days
df_test[f'{last_type}_auth0_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth0_cat0_purchase_date_max']).dt.days
df_test[f'{last_type}_auth0_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth0_cat1_purchase_date_max']).dt.days

# cat0, auth1, auth0の間の期間
df_train[f'{last_type}_auth1_0_term'] = df_train[f'{last_type}_auth1_cat0_term_from_now'] - df_train[f'{last_type}_auth0_cat0_term_from_now']
df_test[f'{last_type}_auth1_0_term']  = df_test[f'{last_type}_auth1_cat0_term_from_now'] - df_test[f'{last_type}_auth0_cat0_term_from_now']

if last_type=='lag0_-2':
    df_train['unix_first_active_month'] =  df_train['first_active_month'].map(lambda x: x.timestamp())
    df_test['first_active_month'].fillna('2017-10', inplace=True)
    df_test['first_active_month'] =  pd.to_datetime(df_test['first_active_month'])
    df_test['unix_first_active_month'] =  df_test['first_active_month'].map(lambda x: x.timestamp())

In [34]:
# Null Impute
for col in df_train.columns:
    if col in ignore_list:continue
    if str(type(df_test[col].dtypes)).count('time'):
        df_train[col].fillna(df_train[col].min(), inplace=True)
        df_test[col].fillna(df_train[col].min(), inplace=True)

### Save Feature

In [35]:
fname = '165_fe3'
fname = '167_fe3'
ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('purchase_date')):continue
    feat_train = df_train[col].astype('float32').values
    try:
        feat_test = df_test[col].astype('float32').values
    except TypeError:
        print(col)
        sys.exit()
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=feat_train)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=feat_test)

In [71]:
import glob
path_list = glob.glob('../features/1_first_valid/*.gz')
for path in path_list:
    length = utils.read_pkl_gzip(path).shape[0]
    if path.count('test'):
        print(length)
#         if length>220000:
#             print(path)
#     if path.count('test'):
#         if length>140000:
#             print(path)

SystemExit: 